### Создание приложения и базы данных

#### Импортируем все, что нужно и создаем приложение

In [1]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3
import random
import string

In [2]:
app = Flask(__name__)

#### Создаем базы данных

База данных для регистраций

In [3]:

conn = sqlite3.connect('participants.db')
c = conn.cursor()

c.execute("""CREATE TABLE answers (
    id INTEGER PRIMARY KEY,
    answers TEXT)
""")

c.execute("""CREATE TABLE participants (
    id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    age INTEGER,
    occupation TEXT,
    code TEXT UNIQUE,
    used INTEGER DEFAULT 0, 
    created TEXT,
    used_at TEXT,
    answer_id INTEGER,
    FOREIGN KEY (answer_id) REFERENCES answers(id))
""")


conn.commit()
conn.close()


### Создание страниц

#### Главная страница

In [4]:
@app.route('/') 
def index():
    return """
    <html><body><p>Привет! Мы проводим лингвистические эксперименты и дарим бесплатный кофе за их прохождение! <br /> 
    Пожалуйста, пройдите нашу небольшую <a href="http://127.0.0.1:5000/register">анкету</a>:) </p>
    
    </body></html>"""

#### Форма для регистрации + анкета

In [5]:
@app.route('/participants', methods=['GET', 'POST'])
def participants():
    if request.method == 'GET':
    #заранее скажу, что все штуки, связанные со шрифтами, помещением текста на страницу
    #методы required и button и тег label я гуглила или спрашивала у нейросетей, потому что 
    #я совсем совсем не знаю как работает html 
    #я свободно могу объяснить любой из этих методов, но я к тому, 
    #что здесь может быть что-то, чего мы очень не брали на парах
    #то же для остальных страниц
        return """
        <html>
        <head>
            <title>Регистрация</title>
            <style>
                body { font-family: Arial; margin: 50px; }
                input, textarea, select { width: 300px; padding: 5px; margin: 5px; }
                button { padding: 10px 20px; margin-top: 20px; }
            </style>
        </head>
        <body>
            <h1>Форма регистрации</h1>
            
            <form method="post">
                <h3>О вас:</h3>
            
            <label>Ваше имя:</label><br><br>
            <h5><i>Введите, пожалуйста, только ИМЯ</i></h5>
            <input type="text" name="first_name" required><br><br>
            
            <label>Ваша фамилия:</label><br><br>
            <h5><i>Введите, пожалуйста, только ФАМИЛИЮ</i></h5><br>
            <input type="text" name="last_name" required><br><br><br>
            
            <label>Возраст:</label><br><br>
            <h5><i>Введите, пожалуйста, число без лишних символов</i></h5><br>
            <input type="number" name="age" required><br><br><br>

            <label>Электронная почта:</label><br> <br>
            <h5><i>Все данные конфиденциальны</i></h5><br>
            <input type="number" name="age" required><br><br><br>
            
            <label>Программа/курс или должность:</label><br><br>
            <h5><i>Введите, пожалуйста, только ИМЯ</i></h5><br>
            <input type="text" name="occupation" required><br><br><br> 
            
                <button type="submit">Перейти к анкете</button>
            </form>
            
            <p><a href="/">Вернуться на главную страницу</a></p>
        </body>
        </html>
        """
    
    else:
        
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        age = request.form['age']
        occupation = request.form['occupation']
        
        #вот эта штука с кодом, про которую писала в первой ячейке
        while True:
            code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
            conn = sqlite3.connect('participants.db')
            c = conn.cursor()
            c.execute("SELECT COUNT(*) FROM participants WHERE code = ?", (code,))
            count = c.fetchone()[0]
            conn.close()
            
            if count == 0: 
                break
        
        conn = sqlite3.connect('participants.db')
        c = conn.cursor()
        c.execute("INSERT INTO participants (first_name, last_name, age, occupation, code) VALUES (?, ?, ?, ?, ?)",
                  (first_name, last_name, age, occupation, code))
        participant_id = c.lastrowid  
        conn.commit()
        conn.close()
        
        return redirect(url_for('enquete', participant_id=participant_id))
        
#ну у нас было требование сделать три страницы, но я не понимаю, как сделать две формы, не создав четвертую страницу...

@app.route('/enquete/<int:participant_id>', methods=['GET', 'POST'])
def enquete(participant_id):
    if request.method == 'GET':
        #тут переусложнила по-моему, но зато какими-то понятными методами...
        return f"""
        <html>
        <body>
            <h1>Лингвистические вопросы</h1>
            <p>Пожалуйста, ответьте на несколько вопросов:</p>
            
            <form method="post">
                <input type="hidden" name="participant_id" value="{participant_id}">
                
                <label>1. Как часто вы думаете об Алексее Игоревиче Виняре?</label><br>
                <textarea name="answer1" rows="3" cols="50" required></textarea><br><br>
                
                <label>2. Верно ли, что Эрика Зотова – лучшая в истории ассистентка линданных?</label><br>
                <textarea name="answer2" rows="3" cols="50" required></textarea><br><br>

                <label>3. Ненавидите ли вы компар так же, как ненавижу его я?</label><br>
                <textarea name="answer2" rows="3" cols="50" required></textarea><br><br>
                
                <button type="submit">Завершить опрос</button>
            </form>
        </body>
        </html>
        """
    
    else: 
        answer1 = request.form['answer1']
        answer2 = request.form['answer2']
        
        answers_json = json.dumps({
        "vinyar": answer1,
        "lingdata": answer2,
        "compar": answer3
        }, ensure_ascii=False)

        conn = sqlite3.connect('participants.db')
        c = conn.cursor()

        c.execute("INSERT INTO answers (answers) VALUES (?)", (answers_json,))
        answer_id = c.lastrowid

        c.execute("UPDATE participants SET answer_id = ? WHERE id = ?",
                  (answer_id, participant_id))

        conn.commit()

        c.execute("SELECT code FROM participants WHERE id = ?", (participant_id,))
        code = c.fetchone()[0]
        conn.close()
        
        #и вот тоже я не уверена, что тут нельзя было сделать более просто, чем через ф-строку
        return f"""
        <html>
        <head>
            <style>
                body {{ font-family: Arial; margin: 50px; text-align: center; }}
                .code {{ font-size: 32px; color: #d35400; font-weight: bold; 
                         padding: 20px; background: #f8f9fa; display: inline-block;
                         margin: 20px; border-radius: 10px; }}
            </style>
        </head>
        <body>
            <p>Спасибо за участие в нашем эксперименте!</p>
            <p>Ваш уникальный код для получения бесплатного кофе:</p>
            <div class="code">{code}</div>
            <p>Вы можете показать этот код бариста в университетском буфете:)</p>
            <p>Учтите, что код можно использовать только один раз!</p>
            <br>
            <p><a href="/">Вернуться на главную</a></p>
        </body>
        </html>
        """



#### Интерфейс для бариста

In [6]:
@app.route('/barista', methods=['GET', 'POST'])
def barista():
    if request.method == 'GET':
        return """
        <html>
        <body>
            <h1>Здесь можно проверить код</h1>
            <form method="post">
                <b>Введите код участника:</b><br>
                <input type="text" name="code" required style="font-size: 20px; padding: 10px;">
                <button type="submit">Проверить</button>
            </form>
            <p><a href="/">На главную</a></p>
        </body>
        </html>
        """
    else:
        code = request.form['code'].strip().upper()
        
        conn = sqlite3.connect('participants.db')
        c = conn.cursor()
        
        c.execute("SELECT first_name, last_name, used FROM participants WHERE code = ?", (code,))
        result = c.fetchone()
        
        if not result:
            conn.close()
            return f"""
            <html><body>
            <h2>Кода {code} не существует</h2>
            <p>Мы не нашли код {code}:( Пожалуйста, проверьте правильность кода!</p>
            <p><a href="/barista">Вернуться на главную</a></p>
            </body></html>
            """
        
        first_name, last_name, used = result 
        #дипсик сказал мне, что эту проверку можно было сделать более правильным, 
        #но более сложным способом, и я решила так не делать
        
        if used == 1:
            conn.close()
            return f"""
            <html><body>
            <h2>Код {code} невалиден</h2>
            <p>Код {code} уже был использован кем-то:(</p>
            <p><a href="/barista">Вернуться на главную</a></p>
            </body></html>
            """
        
        c.execute("UPDATE participants SET used = 1 WHERE code = ?", (code,))
        conn.commit()
        conn.close()
        
        return f"""
        <html><body>
        <h2>Код {code} подтвержден!</h2>
        <p>Участник: {first_name} {last_name}</p>
        <p>Код {code} действителен. Участнику {first_name} {last_name} можно выдать кофе:))</p>
        <p><a href="/barista">Вернуться на главную и проверить другой код</a></p>
        </body></html>
        """

#### Интерфейс для админа

In [7]:
@app.route('/admin')
def admin():
    conn = sqlite3.connect('participants.db')
    c = conn.cursor()
    #дальеш я ссылаюсь только на одну из сделанных таблиц, потому что ну в задании не сказано ничего про часть бд с ответами
    
    c.execute("SELECT COUNT(*) FROM participants")
    number = c.fetchone()[0]
    
    c.execute("SELECT COUNT(*) FROM participants WHERE used = 1")
    used = c.fetchone()[0]

    c.execute("SELECT id, first_name, last_name, age, occupation, code, used FROM participants ORDER BY id")
    participants = c.fetchall()
    
    conn.close()
    
    table_rows = ""
    for p in participants:
        used_text = "Код использован" if p[6] == 1 else "Код пока не использован"
        table_rows += f"""
        <tr>
            <td>{p[0]}</td>
            <td>{p[1]} {p[2]}</td>
            <td>{p[3]}</td>
            <td>{p[4]}</td>
            <td><code>{p[5]}</code></td>
            <td>{used_text}</td>
        </tr>
        """
    
    return f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial; margin: 50px; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 10px; text-align: left; }}
            th {{ background: #f2f2f2; }}
            .stats {{ background: #e8f4f8; padding: 15px; margin: 20px 0; }}
        </style>
    </head>
    <body>
        <h1>Панель исследователя</h1>
        
        <div class="stats">
            <h3>Статистика</h3>
            <p>Количество участников <strong>{number}</strong></p>
            <p>Количество использованных кодов: <strong>{used}</strong></p>
            <p>Количество не использованных кодов: <strong>{number - used}</strong></p>
        </div>
        
        <h3>Список участников:</h3>
        <table>
            <tr>
                <th>ID</th>
                <th>ФИО</th>
                <th>Возраст</th>
                <th>Программа/курс/должность</th>
                <th>Код</th>
                <th>Статус</th>
            </tr>
            {table_rows}
        </table>
        
        <p><a href="/">На главную</a></p>
    </body>
    </html>
    """

In [ ]:
if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Dec/2025 22:15:20] "GET /admin HTTP/1.1" 200 -
